In [3]:
!apt-get -qq update > /tmp/apt.out
!apt-get install -y -qq openjdk-11-jdk-headless
!(wget -q --show-progress -nc https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz)
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
try:
  import pyspark, findspark, delta
except:
  %pip install -q --upgrade pyspark==3.2.1
  %pip install -q findspark
  %pip install -q delta

In [4]:
!pip install gcsfs

In [5]:
!(wget -O /content/spark-3.2.1-bin-hadoop3.2/jars/gcs-connector-hadoop2-latest.jar  -q https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3.2-latest.jar)

In [6]:
import findspark
import pyspark
import os
from pyspark.sql.functions import *

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

findspark.init()
MAX_MEMORY="8g"

spark = (pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .getOrCreate()
    )

spark

In [ ]:
#spark.stop()

In [7]:
!wget https://storage.googleapis.com/bigdata_training/Titanic.parquet

--2024-03-13 16:01:25--  https://storage.googleapis.com/bigdata_training/Titanic.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.145.207, 172.217.219.207, 209.85.146.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.145.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40013 (39K) [application/octet-stream]
Saving to: ‘Titanic.parquet’

Titanic.parquet     100%[===================>]  39.08K  --.-KB/s    in 0.02s   

2024-03-13 16:01:26 (1.69 MB/s) - ‘Titanic.parquet’ saved [40013/40013]



In [8]:
df_passengers = spark.read.parquet("Titanic.parquet")
df_passengers.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [15]:
#Gemiden kurtulma durumuna göre ödenen en yüksek ve en düşük bilet fiyatını bulunuz.

df_passengers.groupBy("Survived").agg(max(col("Fare")),min(col("Fare"))).show()



+--------+---------+---------+
|Survived|max(Fare)|min(Fare)|
+--------+---------+---------+
|       0|    263.0|      0.0|
|       1| 512.3292|      0.0|
+--------+---------+---------+



In [ ]:
#Yolcuların(18 yaş ve altı çocuklar hariç) cinsiyete göre ortalama yaşlarını bulunuz.

df_passengers.where(col("Age")>18).groupBy("Sex").agg(avg(col("Age"))).show()

+------+------------------+
|   Sex|          avg(Age)|
+------+------------------+
|female| 33.90673575129534|
|  male|34.480366492146594|
+------+------------------+



In [16]:
#Aynı kabinde yolculuk yapan erkek ve kadınların eşleşmesi yapılması için aşağıdaki sorgu yazılmışmış fakat sorguda ufak bir eksiklik var, eksik olan parçayı tamamlayınız.
df_passengers.alias("m") \
.join(df_passengers.alias("f"),
      on=((col("m.Cabin")==col("f.Cabin"))),
      how="inner") \
.where((col("m.Sex")=="male")&(col("f.Sex")=="female")) \
.select(col("m.Cabin"),col("f.Name").alias("Kadın Yolcu"),col("m.Name").alias("Erkek Yolcu")).show(100,200)

+-----------+----------------------------------------------------------------------------------+------------------------------------------+
|      Cabin|                                                                       Kadın Yolcu|                               Erkek Yolcu|
+-----------+----------------------------------------------------------------------------------+------------------------------------------+
|C23 C25 C27|                                                    Fortune, Miss. Alice Elizabeth|            Fortune, Mr. Charles Alexander|
|C23 C25 C27|                                                        Fortune, Miss. Mabel Helen|            Fortune, Mr. Charles Alexander|
|        C83|                                      Harris, Mrs. Henry Birkhardt (Irene Wallach)|               Harris, Mr. Henry Birkhardt|
|    B58 B60|                                   Baxter, Mrs. James (Helene DeLaudeniere Chaput)|                  Baxter, Mr. Quigg Edmond|
|       C123|       

In [17]:
#veriye yolcunun yaşına göre belirlenecek yeni bir kolon eklenmek isteniyor(AgeCategory). Kolon içeriğinde;
#0-10 yaş arası yolcular -> Çocuk
#11-18 yaş arası yolcular -> Genç
#19 ve üstü yolcular -> Yetişkin
#bilgisi yazılması isteniyor. Bu yeni kolonu ekleyiniz.

df_passengers.withColumn("AgeCategory",\
                         when((col("Age")>=0) & (col("Age")<=10), lit("Çocuk"))\
                         .when((col("Age")>=11) & (col("Age")<=18), lit("Genç"))\
                         .when(col("Age")>18, lit("Yetişkin"))\
                         .otherwise(lit("Unknown"))).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|AgeCategory|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   Yetişkin|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Yetişkin|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|   Yetişkin|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|   Yetişkin|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|      

In [26]:
#spark'ta bir dataframe sorgu içnide de kullanılabilir. AgeCategory(bir üst sorunın çıktısı olan kolon) kolonuna göre yolcu sayısını veren SQL sorgunu düzenleyiniz.
df_passengers.createOrReplaceTempView("passengers")
spark.sql("SELECT AgeCategory, COUNT(*) as PassengersByAge FROM(  \
      SELECT  \
        CASE  \
            when Age >= 0 AND Age <= 10 then 'Çocuk'  \
            when Age >= 11 AND Age <= 18 then 'Genç'  \
            when Age > 18 then 'Yetişkin'  \
            else 'Unknown'  \
        END AS AgeCategory  \
    FROM passengers)  \
    GROUP BY AgeCategory").show()

+-----------+---------------+
|AgeCategory|PassengersByAge|
+-----------+---------------+
|    Unknown|            177|
|       Genç|             75|
|      Çocuk|             64|
|   Yetişkin|            575|
+-----------+---------------+

